# Notebook to demonstrate the climate tools available
#  Based on data retreived via the cdsapi, you will need a cdsapi key to download the data

In [1]:
# Imports
import os
import xarray
import cdsapi

import tyto

In [2]:
# Create directories 
if not os.path.isdir('test_data'): os.mkdir('test_data')

# set constants
YEARS = [2015]

In [8]:
# Download 5 years of hourly ERA5 data for europe
c = cdsapi.Client()
out_fnames = []
for year in YEARS:
    out_fnames += [f"test_data/era5_temperature_europe_{year}.grib"]
    if not os.path.isfile(out_fnames[-1]):
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': '2m_temperature',
                'year': year,
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '06:00', '12:00', '18:00',
                ],
                'area': [
                    80, -10, 30, 60,
                ],
                'format': 'grib',
            },
        ).download(target=out_fnames[-1])



test_data/era5_temperature_europe_2015.grib


2022-10-17 10:17:09,935 INFO Welcome to the CDS
2022-10-17 10:17:09,935 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-10-17 10:17:10,039 INFO Request is queued
2022-10-17 10:17:11,092 INFO Request is running
2022-10-17 10:18:49,776 WARNING Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Read timed out. (read timeout=60)], attemps 0 of 500
2022-10-17 10:18:49,777 WARNING Retrying in 120 seconds
2022-10-17 10:20:49,781 INFO Retrying now...


In [ ]:
# Open data in xarray
t2m_data = xr.open_mfdataset(out_fnames)